In [ ]:
import os

import numpy as np
import h5py
from matplotlib import pyplot as plt
from astropy import units as u
import time
import pandas as pd

import disco
from disco import readers

import cupy 
cupy.cuda.runtime.setDevice(3)

import utils

# Load Regridded File

In [ ]:
dataset = readers.SwmfOutFieldModelDataset(
    '/home/dedasilv/swmf_test_data2/3d__var_4_t00032600_n00024087.out',
    plasma_params=True,
    grid_downsample=1
)
field_model = dataset[0].duplicate_in_time()

In [ ]:
field_model.extra_fields.keys()

# Make ParticleState

In [ ]:
disp_sim = utils.setup_dispersion_sim()

In [ ]:
disp_sim.particle_state.x.shape

In [ ]:
plt.plot(disp_sim.particle_state.x.get(), disp_sim.particle_state.z.get(), '.')
plt.gca().set_aspect('equal')

# Make a TraceConfig

In [ ]:
config = disco.TraceConfig(
    t_final=-5 * u.hr,
    rtol=1e-3,
    integrate_backwards=True,
    iters_max=10000,
    output_freq=1,
)

# Perform Traces

In [ ]:
fname = 'triple_dispersion.h5'

if os.path.exists(fname):
    hist = disco.ParticleHistory.load(fname)
else:
    start_time = time.time()
    hist = disco.trace_trajectory(config, disp_sim.particle_state, field_model, verbose=1)
    print('took', time.time() - start_time, 's')
    hist.save(fname)
    print(f"Wrote {fname}")

# Plotting

In [ ]:
current_density = np.log10(np.sqrt(field_model.extra_fields['jx']**2 + field_model.extra_fields['jy']**2 + field_model.extra_fields['jz']**2))

In [ ]:

ax = hist.plot_xz(sample=500)
i = field_model.axes.y.value.searchsorted(0)
im = ax.pcolor(field_model.axes.x.value, field_model.axes.z.value, current_density[:, i, :, 0].T)
ax.set_xlim([-30, 30])
ax.set_ylim([-30, 30])
plt.colorbar(im).set_label('Current Density (log)')

In [ ]:
ax = hist.plot_xy(sample=500)
i = field_model.axes.z.value.searchsorted(0)
im = ax.pcolor(field_model.axes.x.value, field_model.axes.y.value, current_density[:, :, i, 0].T)
#ax.set_xlim([-30, 30])
#ax.set_ylim([-30, 30])
plt.colorbar(im).set_label('Current Density (log)')